In [1]:
import scapy.all as sc
import numpy as np
import pandas as pd
from scipy.stats import entropy

In [26]:
SNIFF_TIMEOUT = 1200
BROADCAST = "BROADCAST"
UNICAST = "UNICAST"
PCAP_FILE = "packages-mg-1.pcap"
ARP_OP_WHO_HAS = 1
ARP_OP_IS_AT = 2

## Hago sniff durante sniff_timeout segundos

In [ ]:
%%time
packages = sc.sniff(timeout=SNIFF_TIMEOUT)

## Guardo los paquetes en formato pcap

In [ ]:
sc.wrpcap(PCAP_FILE,packages)

## Leo los paquetes del archivo .pcap

In [11]:
%%time
packages = sc.sniff(offline=PCAP_FILE)
print "Cantidad de paquetes leidos =", len(packages)

Cantidad de paquetes leidos = 46163
CPU times: user 31.3 s, sys: 626 ms, total: 31.9 s
Wall time: 32 s


## Ejercicio 3.2.1

In [12]:
def simbolo_s(package):
    if p.dst == 'ff:ff:ff:ff:ff:ff':
        return BROADCAST
    else:
        return UNICAST

In [13]:
def entropia_fuente(fuente):
    counts = fuente.value_counts()
    return entropy(base=2,pk=counts.as_matrix())

In [14]:
def informacion_simbolos(fuente):
    counts = fuente.value_counts()
    total = counts.sum()
    probs = counts / float(total)
    return pd.Series(- np.log2(probs), index=counts.index)

In [15]:
fuente = pd.Series([simbolo_s(p) for p in packages])
contadores = fuente.value_counts()
print "Entropia fuente S =", entropia_fuente(fuente)
print contadores
print informacion_simbolos(fuente)

Entropia fuente S = 0.0988459488982
UNICAST      45572
BROADCAST      591
dtype: int64
UNICAST      0.018589
BROADCAST    6.287435
dtype: float64


## Ejercicio 3.2.2

In [16]:
arp_packages = sc.sniff(offline=PCAP_FILE, lfilter=lambda x: x.haslayer(sc.ARP))
print "arp packages =", len(arp_packages)

arp packages = 38


In [17]:
#arp_packages = map(lambda p: p["ARP"], arp_packages)
who_has_packages = filter(lambda p : p["ARP"].op == ARP_OP_WHO_HAS,arp_packages)
is_at_packages = filter(lambda p : p["ARP"].op == ARP_OP_IS_AT, arp_packages)
print "who-has packages =",  len(who_has_packages)
print "is-at packages =", len(is_at_packages)

who-has packages = 30
is-at packages = 8


In [19]:
is_at_packages[0]

<Ether  dst=c4:85:08:d9:07:e2 src=c0:4a:00:a7:3d:80 type=0x806 |<ARP  hwtype=0x1 ptype=0x800 hwlen=6 plen=4 op=is-at hwsrc=c0:4a:00:a7:3d:80 psrc=192.168.1.1 hwdst=c4:85:08:d9:07:e2 pdst=192.168.1.110 |<Padding  load='\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' |>>>

In [20]:
who_has_packages[5]

<Ether  dst=ff:ff:ff:ff:ff:ff src=00:e0:4d:58:f4:c8 type=0x806 |<ARP  hwtype=0x1 ptype=0x800 hwlen=6 plen=4 op=who-has hwsrc=00:e0:4d:58:f4:c8 psrc=192.168.1.200 hwdst=00:00:00:00:00:00 pdst=192.168.1.92 |<Padding  load='\x01i\xe0\x00\x00\x02\x94\x04\x00\x00\x17\x00\x00\x00\xe0\x00\x00\xfb' |>>>

## Analisis de simbolos distinguidos utilizando who-has packages

In [21]:
def simbolo_s2(package):
    return package.pdst

In [22]:
fuente_s2 = pd.Series([simbolo_s2(p) for p in who_has_packages])
contadores = fuente_s2.value_counts()
print contadores
print "Entropia de la fuente =", entropia_fuente(fuente_s2)
print "Informacion simbolos"
print informacion_simbolos(fuente_s2)

192.168.1.93     7
192.168.1.1      6
192.168.1.110    5
192.168.1.92     5
192.168.1.222    4
192.168.1.101    2
192.168.1.112    1
dtype: int64
Entropia de la fuente = 2.62753924869
Informacion simbolos
192.168.1.93     2.099536
192.168.1.1      2.321928
192.168.1.110    2.584963
192.168.1.92     2.584963
192.168.1.222    2.906891
192.168.1.101    3.906891
192.168.1.112    4.906891
dtype: float64


## Analisis de simbolos distinguidos utilizando is_at packages

In [23]:
def simbolo_s3(package):
    return package.psrc

In [24]:
fuente_s3 = pd.Series([simbolo_s3(p) for p in is_at_packages])
contadores_s3 = fuente_s3.value_counts()
print contadores_s3
print "Entropia de la fuente =", entropia_fuente(fuente_s3)
print informacion_simbolos(fuente_s3)

192.168.1.110    5
192.168.1.1      3
dtype: int64
Entropia de la fuente = 0.954434002925
192.168.1.110    0.678072
192.168.1.1      1.415037
dtype: float64


In [25]:
len(arp_packages)

38

In [16]:
sc.sniff?